In [47]:
import pandas as p

In [48]:
tree = p.read_excel('tree_te.xlsx')
gru = p.read_excel('gru_te.xlsx')
lstm = p.read_excel('lstm_te.xlsx')
bert = p.read_excel('kcbert_te.xlsx')

In [50]:
bucket = [tree["comment"], tree["y_pred"], gru["predict_label_gru"], 
          lstm["predict_label_lstm"], bert["kcbert_label"]]

In [51]:
bucket = p.DataFrame({'comment': tree['comment'], 'tree': tree['y_pred'], 'gru': gru['predict_label_gru'],
                     'lstm': lstm['predict_label_lstm'], 'bert': bert['kcbert_label'], 'real': bert['real_label']})


In [52]:
bucket

,comment,tree,gru,lstm,bert,real
0,저딴것들이 우리세금 냠냠하네,0,0,1,0,1
1,요즘 대학교 에타보면 저런 애들이 너무많아서 겁이남 학벌주의에 심취하는건 새내기때에...,1,0,0,0,0
2,자기 딸이 회사에서 밥 짓는다고 그러면 화낼 인간들이 직원에게는 저러고 있네 어휴 ...,0,0,0,0,0
3,테이저건 쏘고 묶어놔야지 승무원들 처벌해라 뭐하는거야,0,0,0,0,0
4,네 사정이야 어찌되었든 과속하셨구요 공권력 사칭으로 간주 벌금 이빠이 되겠습니다 서...,1,1,1,0,1
...,...,...,...,...,...,...
39995,저런데 한국이 중국을 도와줘야 되는가 절대 않되는거지 인권이 심각하다 북한이랑 다를...,0,0,0,0,0
39996,이런분들은 영웅이십니다 국민영웅들입니다 이런분들은 표창을 줘야합니다 그런분들 앞으로...,1,1,1,1,1
39997,정신병있네 똑똑한척,1,0,0,0,0
39998,훌륭한 젊은이 아직 대한민국은 그대와같은 젊은이가 있는한 최고가 될수있다 젊은 영웅...,1,1,1,1,1


In [54]:
# 각 모델의 정확도는 아래와 같음(test data 기준)
# tree: 0.762775, gru: 0.8676, lstm: 0.8676, bert: 0.846725

# 각 모델이 vointg에서 가지는 지분을 세팅
tree_power = 0.762
gru_power = 0.867
lstm_power = 0.867
bert_power = 0.846

# 투표 결과를 담아둘 배열
outcome = []

# 지지율 저장할 변수 세팅
pos = 0
neg = 0
winner = 0

for i in range(0, len(bucket['comment'])):
    # tree 투표 진행
    if bucket["tree"][i] == 1: # tree가 1을 지지할 경우
        pos += tree_power # pos는 tree의 지분만큼 지지 얻음
    else:
        neg += tree_power # 그렇지 않다면, neg는 tree의 지분만큼 지지 얻음
    # gru 투표 진행
    if bucket["gru"][i] == 1:
        pos += gru_power
    else:
        neg += gru_power
    # lstm 투표 진행
    if bucket["lstm"][i] == 1: 
        pos += lstm_power
    else:
        neg += lstm_power 
    # bert 투표 진행
    if bucket["bert"][i] == 1:
        pos += bert_power 
    else:
        neg += bert_power 
        
    # 개표
    if pos > neg:
        winner = 1 # pos의 지지율이 더 높다면 real label은 '1'로 판정
    else:
        winner = 0 # 그렇지 않다면 '0'으로 판정
        
    # 개표 결과 저장
    outcome.append(winner)
    
    # 지지율 초기화
    pos = 0
    neg = 0
        
    
bucket.insert(6, 'winner', outcome)
    

In [55]:
# 선거 결과 포함하여 엑셀 파일로 저장
bucket.to_excel(excel_writer = '선거결과(te).xlsx')

In [62]:
# 앙상블로 증가된 정확도 살펴보기
len(bucket['comment'])

# real label과 winner가 일치하는 경우의 수
true_match = 0

for i in range(0, len(bucket['comment'])):
    if bucket['real'][i] == bucket['winner'][i]:
        true_match += 1


In [64]:
true_match

34798

In [66]:
true_match/len(bucket['comment'])

0.86995